## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-02-16-14 +0000,bbc,Bank of England prepared to cut rates if job m...,https://www.bbc.com/news/articles/cqx2pj42395o
1,2025-07-14-02-02-00 +0000,wsj,Singapore’s Stronger-Than-Expected 2Q Advance ...,https://www.wsj.com/finance/currencies/asian-c...
2,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
3,2025-07-14-01-28-07 +0000,nypost,Kristi Noem clashes with NBC anchor Kristen We...,https://nypost.com/2025/07/13/media/kristi-noe...
4,2025-07-14-01-15-53 +0000,bbc,"'If I wasn't here, people could die': Trump pu...",https://www.bbc.com/news/articles/c20w51rkw4no


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,15
109,against,5
106,mamdani,5
56,will,4
65,he,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...,41
5,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...,36
22,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...,33
9,2025-07-13-22-44-12 +0000,nypost,Ex-Secret Service director fires back at Rand ...,https://nypost.com/2025/07/13/us-news/ex-secre...,32
37,2025-07-13-12-58-08 +0000,nypost,President Trump reveals what he really wants h...,https://nypost.com/2025/07/13/us-news/presiden...,32


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,41,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
10,29,2025-07-13-22-43-30 +0000,nypost,Ex-AOC top aide urges Mamdani to run primaries...,https://nypost.com/2025/07/13/us-news/ex-aoc-a...
5,21,2025-07-14-01-05-35 +0000,nypost,Trump announces US will send Patriot missiles ...,https://nypost.com/2025/07/13/world-news/trump...
22,18,2025-07-13-19-11-43 +0000,nypost,President Trump and first lady Melania arrive ...,https://nypost.com/2025/07/13/us-news/presiden...
24,17,2025-07-13-18-26-53 +0000,nypost,"Megyn Kelly, MAGA stars call for Pam Bondi’s h...",https://nypost.com/2025/07/13/us-news/megyn-ke...
45,16,2025-07-13-02-44-25 +0000,nypost,Border czar Tom Homan delivers fiery response ...,https://nypost.com/2025/07/12/us-news/border-c...
9,15,2025-07-13-22-44-12 +0000,nypost,Ex-Secret Service director fires back at Rand ...,https://nypost.com/2025/07/13/us-news/ex-secre...
33,15,2025-07-13-15-20-08 +0000,bbc,Texas camp buildings were removed from map sho...,https://www.bbc.com/news/articles/ce8zjk5yx8wo
6,12,2025-07-14-00-54-28 +0000,nypost,Trump quips about signing executive order mand...,https://nypost.com/2025/07/13/us-news/trump-qu...
13,12,2025-07-13-22-20-06 +0000,bbc,Two women killed at Kentucky church as gunman ...,https://www.bbc.com/news/articles/cm2l5jn6254o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
